Instructions to install everything: https://jckantor.github.io/ND-Pyomo-Cookbook/01.01-Installing-Pyomo.html

In [1]:
import pyomo.environ as pyo
import pandas as pd

## Import Data

In [2]:
initial_positions = pd.read_csv("positions.csv")
initial_positions.head()

,Unnamed: 0,id,x,y,z
0,0,0,0.942906,-0.074538,0.026742
1,1,1,2.454963,-0.074537,0.026741
2,2,2,0.558596,0.949435,0.054739
3,3,3,0.558596,-0.610770,0.899529
4,4,4,0.558596,-0.562277,-0.874044


In [3]:
bond_lengths = pd.read_csv("constraints.csv")
bond_lengths.head()

,Unnamed: 0,id_i,id_j,req
0,0,0,1,1.529
1,1,0,2,1.090
2,2,0,3,1.090
3,3,0,4,1.090
4,4,5,1,1.090


## Create Model

In [4]:
m = pyo.ConcreteModel()

# Create list of atom indicies
m.ATOMS = pyo.Set(initialize=initial_positions.id.tolist())

# Create target values
m.x_start = pyo.Param(m.ATOMS, initialize=initial_positions.x.to_dict(),within=pyo.Reals)
m.y_start = pyo.Param(m.ATOMS, initialize=initial_positions.y.to_dict(),within=pyo.Reals)
m.z_start = pyo.Param(m.ATOMS, initialize=initial_positions.z.to_dict(),within=pyo.Reals)

# Create bond lengths
I = bond_lengths.id_i.tolist()
J = bond_lengths.id_j.tolist()
L = bond_lengths.req.tolist()

# Construct an empty dictionary
B = {}

# Loop over bonds
for n in range(len(I)):
    B[(I[n],J[n])] = L[n]

# Create bond length set
# m.BONDS = pyo.Set(B.keys())    

# Create bond length parameter
m.bond_lengths = pyo.Param(m.ATOMS, m.ATOMS, initialize=B)

# Create position variables
m.x = pyo.Var(m.ATOMS, initialize=initial_positions.x.to_dict(),within=pyo.Reals)
m.y = pyo.Var(m.ATOMS, initialize=initial_positions.y.to_dict(),within=pyo.Reals)
m.z = pyo.Var(m.ATOMS, initialize=initial_positions.z.to_dict(),within=pyo.Reals)

# Add bond length constraint
# TODO: define this over the set BONDS
def calc_bond_length(m, i, j):
    if (i, j) in B.keys():
        return (m.x[i] - m.x[j])**2 + (m.y[i] - m.y[j])**2 + (m.z[i] - m.z[j])**2 == m.bond_lengths[(i,j)]**2
    else:
        return pyo.Constraint.Skip

m.eq_calc_bound_length = pyo.Constraint(m.ATOMS, m.ATOMS, rule=calc_bond_length)


m.obj = pyo.Objective(expr = sum((m.x[i] - m.x_start[i])**2
                                 + (m.y[i] - m.y_start[i])**2 
                                 + (m.z[i] - m.z_start[i])**2 
                                 for i in m.ATOMS))

In [5]:
m.pprint()

3 Set Declarations
    ATOMS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    8 : {0, 1, 2, 3, 4, 5, 6, 7}
    bond_lengths_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain      : Size : Members
        None :     2 : ATOMS*ATOMS :   64 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)}
    eq_calc_bound_length_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain      : Size : Members
        None :     2 : ATOMS*ATOMS 

In [6]:
results = pyo.SolverFactory('ipopt').solve(m,tee=True)

Ipopt 3.12.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       42
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       45

Total number of variables............................:       24
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

In [7]:
x_sln = []
y_sln = []
z_sln = []

for i in m.ATOMS:
    x_sln.append(m.x[i].value)
    y_sln.append(m.x[i].value)
    z_sln.append(m.x[i].value)